<a href="https://colab.research.google.com/github/Aqeel151/Myproject/blob/master/PROJECTS_03_function_calling.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
!pip install -qU langchain langchain-google-genai google-generativeai python-dotenv

In [15]:
!pip install -qU langchain langchain_community langchain_google_genai

In [16]:
import os
from typing import Dict, Any
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import tool
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationChain
from langchain.agents.openai_functions_agent.base import OpenAIFunctionsAgent
from langchain.prompts import MessagesPlaceholder
from langchain.schema.messages import SystemMessage
from dotenv import load_dotenv
from typing import Any
from langchain.tools import BaseTool

In [17]:
# Load environment variables
load_dotenv()
GOOGLE_API_KEY = os.getenv('GOOGLE_API_KEY')

In [18]:
class Calculator:
    """Base Calculator class for arithmetic operations"""
    def calculate(self, expression: str) -> str:
        try:
            # Define safe operations
            safe_dict = {
                'abs': abs,
                'float': float,
                'int': int,
                'max': max,
                'min': min,
                'pow': pow,
                'round': round
            }

            # Clean the expression
            cleaned_expr = ''.join(c for c in expression if c in '0123456789.+-*/() ')

            # Evaluate using safe dictionary
            result = eval(cleaned_expr, {"__builtins__": None}, safe_dict)
            return str(result)
        except Exception as e:
            return f"Error: Invalid expression - {str(e)}"

In [19]:
from typing import Any, Dict

from langchain.tools import BaseTool

class Calculator:
    """Base Calculator class for arithmetic operations"""
    def calculate(self, expression: str) -> str:
        try:
            # Define safe operations
            safe_dict = {
                'abs': abs,
                'float': float,
                'int': int,
                'max': max,
                'min': min,
                'pow': pow,
                'round': round
            }

            # Clean the expression
            cleaned_expr = ''.join(c for c in expression if c in '0123456789.+-*/() ')

            # Evaluate using safe dictionary
            result = eval(cleaned_expr, {"__builtins__": None}, safe_dict)
            return str(result)
        except Exception as e:
            return f"Error: Invalid expression - {str(e)}"

class CalculatorTool(BaseTool):
    """Calculator tool for LangChain"""
    name: str = "calculator"  # Added type annotation here
    description: str = "Useful for performing arithmetic calculations. Input should be a mathematical expression as a string."  # Added type annotation here

    def _run(self, expression: str) -> str:
        """Execute the calculator tool"""
        calc = Calculator()
        return calc.calculate(expression)

    def _arun(self, expression: str) -> str:
        """Async version of the tool (not implemented)"""
        raise NotImplementedError("CalculatorTool does not support async")

In [20]:
def setup_agent():
    """Set up the LangChain agent with the calculator tool"""
    # Initialize LLM
    llm = ChatGoogleGenerativeAI(
        model="gemini-pro",
        google_api_key=GOOGLE_API_KEY,
        temperature=0
    )

    # Assuming these variables are defined somewhere before this function call
    tools = [CalculatorTool()]  # Example definition of tools
    system_message = SystemMessage(content="You are a helpful AI assistant.")  # Example system message

    # Create prompt
    prompt = OpenAIFunctionsAgent.create_prompt(
        system_message=system_message,
        extra_prompt_messages=[MessagesPlaceholder(variable_name="memory")]
    )
    # Initialize memory
    memory = ConversationBufferMemory(memory_key="memory", return_messages=True)

    # Create agent
    agent = OpenAIFunctionsAgent(llm=llm, tools=tools, prompt=prompt)

    return agent # Added return statement to return the agent

In [21]:
def setup_agent_executor(agent, tools, memory):  # New function to wrap the agent executor setup
    """Set up and return the LangChain agent executor"""
    return AgentExecutor(
        agent=agent,
        tools=tools,
        memory=memory,
        verbose=True
    )

In [23]:
def run_test_queries():
    """Run test queries on the calculator agent"""
    # Initialize agent
    agent_executor = setup_agent()

    # Run queries
    test_queries = ["What is 2 plus 2?", "Calculate 10 minus 5.", "What is the square root of 9?"]
    for query in test_queries:
        print("\nQuery:", query)
        try:
            response = agent_executor.run(query)
            print("Response:", response)
        except Exception as e:
            print(f"Error processing query: {str(e)}")
        print("-" * 40)

if __name__ == "__main__":
    run_test_queries()


Query: What is 2 plus 2?
Error processing query: 'OpenAIFunctionsAgent' object has no attribute 'run'
----------------------------------------

Query: Calculate 10 minus 5.
Error processing query: 'OpenAIFunctionsAgent' object has no attribute 'run'
----------------------------------------

Query: What is the square root of 9?
Error processing query: 'OpenAIFunctionsAgent' object has no attribute 'run'
----------------------------------------
